# Import Libraries

In [1]:
pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
import sys
sys.path.append('..')

In [3]:
import random
from copy import deepcopy
from lib.Projectsolution import Solution
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [4]:
import sys
sys.path.append('./lib')

from lib.genetic_algorithm import genetic_algorithm
from lib.crossover import pmx_crossover, order_crossover
from lib.mutation import swap_mutation, inversion_mutation, insertion_mutation
from lib.fitness import fitness
from lib.selection import tournament_selection, ranking_selection
from lib.Projectsolution import Solution  


# Import Data

In [6]:
# Load artists data
artists_df = pd.read_csv(r"C:\Users\anama\OneDrive\Ambiente de Trabalho\Mestrado\S2\CIFO\data\artists(in).csv")

# Preview the first few rows
print(artists_df.head())

print(f"\n Number of artists: {len(artists_df)}")

   Unnamed: 0             name       genre  popularity
0           0    Midnight Echo        Rock          75
1           1      Solar Flare  Electronic          78
2           2     Velvet Pulse        Jazz          35
3           3     Neon Reverie  Electronic         100
4           4  The Silver Owls   Classical          85

 Number of artists: 35


In [7]:
# Rename the column
artists_df.rename(columns={"Unnamed: 0": "id"}, inplace=True)

# Confirm the change
print(artists_df.head())

   id             name       genre  popularity
0   0    Midnight Echo        Rock          75
1   1      Solar Flare  Electronic          78
2   2     Velvet Pulse        Jazz          35
3   3     Neon Reverie  Electronic         100
4   4  The Silver Owls   Classical          85


In [8]:
artists_df['genre_code'], genre_uniques = pd.factorize(artists_df['genre'])

print(artists_df[['genre', 'genre_code']].head())

        genre  genre_code
0        Rock           0
1  Electronic           1
2        Jazz           2
3  Electronic           1
4   Classical           3


In [9]:
# Normalize the 'popularity' column 
popularity = artists_df[['popularity']].values  
scaler = MinMaxScaler()
artists_df['popularity_norm'] = scaler.fit_transform(popularity)


In [24]:
class Artist:
    def __init__(self, id, name, popularity, genre, genre_code, popularity_norm):
        self.id = id
        self.name = name
        self.popularity = popularity
        self.genre = genre
        self.genre_code = genre_code
        self.popularity_norm = popularity_norm

    def __repr__(self):
        return f"{self.name} (Pop: {self.popularity}, Genre: {self.genre}, NormPop: {self.popularity_norm})"


# Convert to list of Artist objects
artists = [
    Artist(row["id"], row["name"], row["popularity"], row["genre"], row["genre_code"], row['popularity_norm'])
    for _, row in artists_df.iterrows()
]

In [11]:
# Try to load with headers and index
conflict_df = pd.read_csv(r"C:\Users\anama\OneDrive\Ambiente de Trabalho\Mestrado\S2\CIFO\data\conflicts(in).csv", index_col=0)

# Convert to NumPy matrix for easier computation later
conflict_matrix = conflict_df.to_numpy()

# Check matrix size
print("Conflict matrix shape:", conflict_matrix.shape)

Conflict matrix shape: (35, 35)


In [12]:
conflict_matrix[:5, :5]  

array([[0. , 0. , 0. , 0.2, 0.5],
       [0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 1. , 0.5],
       [0.2, 1. , 1. , 0. , 0.2],
       [0.5, 0. , 0.5, 0.2, 0. ]])

In [13]:
artists_df.head()

,id,name,genre,popularity,genre_code,popularity_norm
0,0,Midnight Echo,Rock,75,0,0.6875
1,1,Solar Flare,Electronic,78,1,0.7250
2,2,Velvet Pulse,Jazz,35,2,0.1875
3,3,Neon Reverie,Electronic,100,1,1.0000
4,4,The Silver Owls,Classical,85,3,0.8125


# Tests

In [19]:
# Create initial population:
artist_ids = list(artists_df['id'])

crossover_method = 'pmx'      # try 'ox' too
mutation_method = 'swap'      # try 'inversion' or 'insertion'
selection_method = tournament_selection  # or ranking_selection

initial_population = [
    Solution(
        random.sample(artist_ids, len(artist_ids)),
        artists=artists,  
        conflict_matrix=conflict_matrix,
        crossover_method=crossover_method,
        mutation_method=mutation_method
    )
    for _ in range(50)
]



In [20]:
best_solution = genetic_algorithm(
    initial_population=initial_population,
    max_gen=100,
    selection_algorithm=selection_method,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.1,
    elitism=True,
    verbose=True,
)


-------------- Generation: 1 --------------
Selected individuals:
[11, 13, 8, 19, 22, 17, 28, 20, 14, 2, 21, 3, 34, 29, 12, 5, 10, 31, 27, 7, 24, 23, 9, 6, 30, 26, 33, 0, 25, 15, 1, 18, 32, 4, 16]
[22, 1, 0, 15, 8, 31, 4, 29, 10, 5, 28, 12, 26, 27, 13, 19, 18, 14, 2, 20, 17, 30, 25, 23, 6, 24, 34, 7, 11, 16, 33, 9, 3, 32, 21]
Applied crossover
Offspring:
[22, 34, 20, 16, 8, 14, 4, 29, 18, 19, 28, 13, 17, 2, 12, 5, 10, 31, 27, 7, 24, 23, 9, 6, 30, 26, 33, 0, 25, 15, 1, 11, 3, 32, 21]
[9, 12, 8, 5, 22, 26, 28, 0, 31, 27, 21, 3, 1, 29, 13, 19, 18, 14, 2, 20, 17, 30, 25, 23, 6, 24, 34, 7, 11, 16, 33, 10, 32, 4, 15]
First mutated individual: [22, 34, 20, 16, 8, 14, 4, 29, 18, 19, 28, 13, 17, 2, 12, 5, 10, 31, 27, 7, 24, 23, 9, 6, 30, 26, 33, 0, 25, 15, 1, 11, 3, 32, 21]
Second mutated individual: [9, 12, 8, 5, 22, 26, 28, 0, 31, 27, 21, 3, 1, 29, 13, 19, 32, 14, 2, 20, 17, 30, 25, 23, 6, 24, 34, 7, 11, 16, 33, 10, 18, 4, 15]
Selected individuals:
[32, 7, 30, 20, 9, 21, 16, 34, 6, 25, 3, 22,

In [26]:
from itertools import product

# Define the methods for testing
crossover_methods = ['pmx', 'ox']
mutation_methods = ['swap', 'inversion', 'insertion']
selection_methods = [tournament_selection, ranking_selection]

# Initialize variables to track the best results
best_fitness = -float('inf')  # Start with a very low value
best_combination = None  # To store the best combination of parameters
best_solution = None  # To store the best solution

# Iterate through all combinations of parameters
for xo, mut, sel in product(crossover_methods, mutation_methods, selection_methods):
    print(f"Testing crossover: {xo}, mutation: {mut}, selection: {sel.__name__}")

    # Initialize the population for the current combination of parameters
    pop = [
        Solution(
            random.sample(artist_ids, len(artist_ids)),
            artists=artists,
            conflict_matrix=conflict_matrix,
            crossover_method=xo,
            mutation_method=mut
        )
        for _ in range(50)
    ]

    # Run the genetic algorithm with the current population and parameters
    current_best_solution = genetic_algorithm(
        initial_population=pop,
        max_gen=50,
        selection_algorithm=sel,
        maximization=True,
        elitism=True,
        verbose=False
    )

    # Get the fitness of the current best solution
    current_fitness = current_best_solution.fitness()

    print(f"Best fitness: {current_fitness:.4f}")

    # If the current fitness is better than the best fitness so far, update the best results
    if current_fitness > best_fitness:
        best_fitness = current_fitness
        best_combination = (xo, mut, sel)
        best_solution = current_best_solution

# After the loop, print the best combination and the best solution
print("\nBest combination of parameters:")
print(f"Crossover: {best_combination[0]}")
print(f"Mutation: {best_combination[1]}")
print(f"Selection: {best_combination[2].__name__}")

# Print the best solution
print("\nBest solution:")
print(best_solution)



Testing crossover: pmx, mutation: swap, selection: tournament_selection
Best fitness: 0.9438
Testing crossover: pmx, mutation: swap, selection: ranking_selection
Best fitness: 0.8637
Testing crossover: pmx, mutation: inversion, selection: tournament_selection
Best fitness: 0.9451
Testing crossover: pmx, mutation: inversion, selection: ranking_selection
Best fitness: 0.8736
Testing crossover: pmx, mutation: insertion, selection: tournament_selection
Best fitness: 0.9306
Testing crossover: pmx, mutation: insertion, selection: ranking_selection
Best fitness: 0.8819
Testing crossover: ox, mutation: swap, selection: tournament_selection
Best fitness: 0.9340
Testing crossover: ox, mutation: swap, selection: ranking_selection
Best fitness: 0.8859
Testing crossover: ox, mutation: inversion, selection: tournament_selection
Best fitness: 0.9238
Testing crossover: ox, mutation: inversion, selection: ranking_selection
Best fitness: 0.8980
Testing crossover: ox, mutation: insertion, selection: tour

In [29]:
# Number of stages and slots
num_stages = 5
num_slots = 7

# Get the best solution (list of indices)
best_artist_solution = [artists[i] for i in best_solution.representation]

# Create a 2D list (5 stages x 7 slots) to represent the schedule
schedule = [[None for _ in range(num_slots)] for _ in range(num_stages)]

# Fill the schedule with the artists based on the best solution
for stage in range(num_stages):
    for slot in range(num_slots):
        index = stage * num_slots + slot
        schedule[stage][slot] = best_artist_solution[index]

# Display the schedule
for stage in range(num_stages):
    print(f"Stage {stage + 1}:")
    for slot in range(num_slots):
        artist = schedule[stage][slot]
        print(f"  Slot {slot + 1}: {artist.name}, Popularity: {artist.popularity}, Genre: {artist.genre}")
    print()


Stage 1:
  Slot 1: Velvet Pulse, Popularity: 35, Genre: Jazz
  Slot 2: Static Mirage, Popularity: 94, Genre: Rock
  Slot 3: Cosmic Frequency, Popularity: 53, Genre: Rock
  Slot 4: The Jazz Nomads, Popularity: 64, Genre: Jazz
  Slot 5: Blue Horizon, Popularity: 51, Genre: Pop
  Slot 6: The Wandering Notes, Popularity: 84, Genre: Jazz
  Slot 7: Rhythm Alchemy, Popularity: 94, Genre: Jazz

Stage 2:
  Slot 1: Phantom Groove, Popularity: 47, Genre: Hip-Hop
  Slot 2: Shadow Cadence, Popularity: 66, Genre: Jazz
  Slot 3: The Polyrhythm Syndicate, Popularity: 66, Genre: Jazz
  Slot 4: Midnight Echo, Popularity: 75, Genre: Rock
  Slot 5: Deep Resonance, Popularity: 90, Genre: Jazz
  Slot 6: Aurora Skies, Popularity: 75, Genre: Pop
  Slot 7: Lunar Spectrum, Popularity: 99, Genre: Rock

Stage 3:
  Slot 1: Mystic Rhythms, Popularity: 78, Genre: Classical
  Slot 2: The Bassline Architects, Popularity: 61, Genre: Hip-Hop
  Slot 3: Nightfall Sonata, Popularity: 84, Genre: Classical
  Slot 4: Neon Rev